# Способы соединения таблиц

## ОПЕРАТОРЫ

### В прошлых юнитах для соединения таблиц мы использовали один оператор — JOIN. В действительности же он не один: существует несколько различных видов соединений (join’ов) — давайте посмотрим, какие ↓

## INNER JOIN

### INNER JOIN — это тот же JOIN (слово inner в операторе можно опустить).

### Принцип работы этого оператора визуализирует изображение ниже.

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4d5d75b4a4f5a5ca654aac79889ffe4c/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_5_1.gif)

### Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

### Рассмотрим на примере нашего датасета. 

### В таблице teams есть данные о 299 различных командах — можем проверить это с помощью запроса.

In [ ]:
'''
SELECT 
COUNT(DISTINCT id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM sql.teams /*таблица teams*/
'''

In [ ]:
'''
SELECT 
COUNT(DISTINCT api_id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец api_id*/
FROM sql.teams
'''

### Теперь добавим к teams таблицу с матчами.

In [ ]:
'''
SELECT 
COUNT(DISTINCT t.id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM 
sql.teams t /*таблица teams с алиасом t*/
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения inner JOIN; таблица teams с алиасом t; 
условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
'''

### И в таблице останется уже не 299 команд, а только 292.

### Почему?

### Дело в том, что таблица sql.matches по какой-то причине не содержит информацию о командах Lierse SK, KVC Westerlo, KAS Eupen, Club Brugge KV, KV Oostende, RSC Anderlecht и Hull City, зато они есть в таблице sql.teams. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

## LEFT OUTER JOIN И RIGHT OUTER JOIN

### Также существуют схожие друг с другом типы соединения — LEFT JOIN и RIGHT JOIN (слово outer в операторе можно опустить).

### Принцип работы оператора можно описать так:

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d81b9a709fc7a2e677adc190862521a0/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_5_2.gif)

### Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

### LEFT JOIN может быть полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

### Почему соответствий может не быть?

### Причины могут быть разные, назовём две основные:

* Различная бизнес-логика таблиц.

Пример: в интернет-магазине почти всегда можно оформить заказ с промокодом. Вполне вероятно, что информация о промокодах хранится в отдельной таблице, но при этом не для каждого заказа будет существовать промокод, поскольку покупку можно оформить и без него.

* Разное время обновления таблиц.

Пример: клиент зарегистрировался на сайте интернет-магазина и оформил заказ. Данные о регистрации клиентов могут обновляться в базе данных раз в неделю, а о заказах — раз в сутки.

### → Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.

### Для начала посмотрим на результат запроса после соединения.

In [ ]:
'''
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    m.id /*столбец id таблицы m*/
FROM sql.teams t /*таблица teams с алиасом t*/
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения left JOIN; 
таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
ORDER BY m.id DESC /*сортировка по id таблицы m по убыванию, чтобы увидеть строки со значением null*/
'''

### Вывод: в таблице teams сохранились все записи, а в таблице matches есть пустые строки.

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/ed01b99d33d792861e03db598554d3bf/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_5_3.png)

### Теперь, чтобы выбрать такие команды, которые не принимали участия в матчах, достаточно добавить условие where m.id is null (или любое другое поле таблицы matches).



In [ ]:
'''
SELECT
    t.long_name /*столбец long_name таблицы t*/
FROM 
    sql.teams t /*таблица teams с алиасом t*/
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения left JOIN; 
таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
WHERE m.id IS NULL /*условие: столбец id таблицы m имеет значение null*/
'''

### Обратите внимание! Если мы добавим какой-либо фильтр по значению для таблицы matches, то LEFT JOIN превратится в INNER JOIN, поскольку для второй таблицы станет необходимым присутствие значения в строке.

In [ ]:
'''
SELECT
    t.long_name /*столбец long_name таблицы t*/
FROM sql.teams t /*таблица teams с алиасом t*/
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения left JOIN; 
таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
WHERE
    m.season = '2008/2009'  /*условие: столбец season таблицы m имеет значение 2008/2009*/
AND t.long_name = 'KAS Eupen' /*условие: столбец long_name таблицы t имеет значение KAS Eupen*/
'''

### В ответе Metabase получим No results!, так как вместе с фильтром m.season = '2008/2009' исчезли все строки, значения которых NULL.

In [ ]:
# 5.1
'''
select
  distinct t.long_name
from sql.matches m
  left join sql.teams t on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
group by t.long_name
'''

### С LEFT JOIN также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.



In [ ]:
'''
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM 
    sql.teams t /*таблица teams с алиасом t*/
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id /*оператор соединения LEFT JOIN; таблица matches с алиасом m; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/
ORDER BY 2 DESC /*сортировка по столбцу total_goals по убыванию, чтобы увидеть строки со значением null*/
'''

## Обратите внимание! При применении функций SUM, MIN, MAX, AVG к полям со значением NULL в результате получится NULL, а не 0. А при использовании функции COUNT, наоборот, получится 0.

In [ ]:
# 5.2
'''
select
  t.long_name,
  count(m.id) matches_cnt
from sql.teams t
  left join sql.matches m on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
group by t.id
order by 2, 1
'''

### При использовании RIGHT JOIN сохраняется та же логика, что и для LEFT JOIN, только за основу берётся правая таблица.

### Чтобы из LEFT JOIN получить RIGHT JOIN, нужно просто поменять порядок соединения таблиц.

### Вообще, применение RIGHT JOIN считается плохим тоном, так как язык SQL читается и пишется слева направо, а такой оператор усложняет чтение запросов.

## FULL OUTER JOIN

### Оператор FULL OUTER JOIN объединяет в себе LEFT и RIGHT JOIN и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

### FULL OUTER JOIN может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

### Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

### Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать FULL OUTER JOIN для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по id пользователя.

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/6ca460197802a26591af38d2f671d8c3/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_5_4.png)

### Синтаксис FULL OUTER JOIN аналогичен другим JOIN.

### SELECT 
### …
### FROM
### 	table1
### FULL OUTER JOIN table2 ON условие

## CROSS JOIN

### На самом деле с этим оператором соединения таблиц вы познакомились в самом начале текущего модуля.

### CROSS JOIN соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/bab33905e69032274d6f7e9f4abced95/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_5_5.png)

### В начале модуля вы использовали этот оператор, записывая таблицы через запятую.



In [ ]:
'''
SELECT * /*выбор всех полей*/
FROM
    sql.teams, /*таблица teams*/
    sql.matches /*таблица matches*/
'''

### Этот же запрос можно записать с использованием CROSS JOIN.

In [ ]:
'''
SELECT * /*выбор всех полей*/
FROM
    sql.teams /*таблица teams*/
    CROSS JOIN sql.matches /*таблица matches*/
'''

## Обратите внимание! Условие для CROSS JOIN, в отличие от других операторов, не требуется.

### Также этот запрос можно записать с помощью INNER JOIN с условием on true — результат будет тот же.

In [ ]:
'''
SELECT * /*выбор всех полей*/
FROM
    sql.teams /*таблица teams*/
    JOIN sql.matches ON TRUE /*оператор соединения INNER JOIN; таблица matches; условие: для всех случаев*/
'''

### CROSS JOIN может быть полезен, когда необходимо создать таблицу фактов.

### Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.

In [ ]:
'''
SELECT
    DISTINCT /*оператор исключения повторяющихся строк*/
    t1.long_name home_team, /*столбец long_name таблицы t1; новое название*/
    t2.long_name away_team /*столбец long_name таблицы t2; новое название*/
FROM
    sql.teams t1 /*таблица teams с алиасом t1*/
    CROSS JOIN sql.teams t2 /*оператор соединения CROSS JOIN; таблица teams с алиасом t2*/
'''

In [ ]:
# 5.3
'''
select
  distinct
  t1.short_name home_team,
  t2.short_name away_team
from sql.teams t1
  cross join sql.teams t2
group by 1, 2
order by 1, 2
'''

## NATURAL JOIN

### Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

## NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

* → NATURAL INNER JOIN (возможна запись NATURAL JOIN);
* → NATURAL LEFT JOIN;
* → NATURAL RIGHT JOIN;
* → NATURAL FULL OUTER JOIN.

### При использовании NATURAL JOIN прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц teams и matches этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец id, но таблицы соединяются по другим столбцам.

### Когда у таблиц есть несколько столбцов с одинаковыми именами, при NATURAL JOIN условие соединения будет применено на все столбцы с одинаковыми именами.

## То есть для таблиц table1 и table2

### table1: id, name, ...

### table2: id, name, ...

## запрос

### SELECT 
### …
### FROM          table1 NATURAL JOIN table2

## будет равнозначен запросу

### SELECT
### …
### FROM          table1 t1
### INNER JOIN table2 t2 ON t1.id = t2.id AND t1.name = t2.name


## ОБЩАЯ ЛОГИКА ПОСТРОЕНИЯ ЗАПРОСА С JOIN

### При построении запроса с несколькими JOIN старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.

### Например, для ответа на вопрос: «Какая команда сыграла больше всех матчей в сезоне 2010/2011?» в качестве центральной лучше выбрать таблицу с командами.

### А для ответа на вопрос: «В каком сезоне участвовало больше всего команд?» — таблицу с матчами.

### Стоит отметить, что из рассмотренных видов соединений чаще всего используются INNER JOIN и LEFT JOIN. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.

## Виды джойнов

## [Здесь!](https://view.genial.ly/617fe0627516f70d8cd39fb0/horizontal-infographic-review-modul-sql-3)

In [ ]:
# 6.1
'''
select
  t.long_name
from sql.teams t
  join sql.matches m on t.api_id = m.away_team_api_id
where season = '2012/2013'
group by 1
order by 1
'''

In [ ]:
# 6.2
'''
select
  t.long_name,
  count(m.id) matches_cnt
from
  sql.teams t
  join sql.matches m on t.api_id = m.home_team_api_id
where t.long_name = 'Inter'
group by 1
order by 2
'''

In [ ]:
# 6.3
'''
select
  t.long_name,
  sum(m.away_team_goals) total_goals
from
  sql.teams t
  join sql.matches m on t.api_id = m.away_team_api_id
group by 1
order by 2 desc
limit 10
'''

In [ ]:
# 6.4
'''
select
  count(t1.id) + count(t2.id)
from
  sql.matches m
  join sql.teams t1 on t1.api_id = m.home_team_api_id 
  join sql.teams t2 on t2.api_id = m.away_team_api_id
where (t1.long_name = 'Real Madrid CF' and t2.long_name = 'FC Barcelona')
order by 1
'''

In [ ]:
# 6.5
'''
select
  t.long_name,
  m.season,
  sum(m.home_team_goals) total_goals
from
  sql.teams t
  join sql.matches m on t.api_id = m.home_team_api_id
group by 1, 2
having sum(m.home_team_goals) < 10
order by 1, 2, 3
'''